In [5]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import os.path
import pickle



def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = None
    creds_filename = '3-3-2daebong-spreadsheet.json'
    # 토큰 파일이 있는지 확인합니다. 있으면 인증 정보를 로드합니다.
    if os.path.exists('token.json'):
        # creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        flow = InstalledAppFlow.from_client_secrets_file(
                creds_filename, SCOPES)
        creds = flow.run_local_server(port=8080)
        # flow = InstalledAppFlow.from_client_secrets_file(
        #         creds_filename, SCOPES)
        # creds = flow.run_local_server(port=8080)
    # 토큰이 없거나 유효하지 않으면 사용자 인증을 다시 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 파일로 저장합니다.
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds


def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [19]:
def apply_protect_data(service, spreadsheet_id):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')

    product_list = None
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            product_list = sheet.get('properties', {}).get('sheetId', '')



    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

    if product_list is not None:
        # 보호된 범위 설정
        requests = [
            {
                'addProtectedRange': {
                    'protectedRange': {
                        'range': {
                            'sheetId': product_list,
                            'startRowIndex': 0,  # 11행부터 시작 (0 기반 인덱스)
                            'startColumnIndex': 0,  # J열 (0 기반 인덱스)
                            # endRowIndex, endColumnIndex를 생략하면 범위의 나머지 부분을 포함합니다.
                        },
                        'description': '상품목록 잠금',
                        'editors': {
                            'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                        }
                    }
                }
            }
        ]

        body = {
            'requests': requests
        }

        response = service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()

In [21]:
def get_sheet_data(service, spreadsheet_id, range_name):
    """특정 스프레드시트의 지정된 범위에 있는 데이터를 가져옵니다."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # if not values:
    #     print('No data found.')
    # else:
    #     for row in values:
    #         # Row 데이터를 출력합니다. 필요에 따라 여기서 다른 작업을 수행할 수 있습니다.
    #         print(row)

    return values
    

In [23]:
signup_spreadsheets_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
RANGE_NAME = '셀러컨택포인트!K206:K'  # 예를 들어 Sheet1의 K2부터 K 끝까지
import time

index = [
0 #226
,3 #229
,7 #233
,8 #234
,10 #236
,12 #238
,13 #239
,15 #241
,17 #243
,27 #253
,28 #254
,31 #257
]

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 

    spreadsheet_ids = get_sheet_data(service_sheets, signup_spreadsheets_id, RANGE_NAME)

    for i in index:
        time.sleep(0.1)
        if len(spreadsheet_ids[i]) > 0:
            # print(i)
            try: 
                apply_protect_data(service_sheets, spreadsheet_ids[i][0])
            except:
                print(i)
                print(spreadsheet_ids[i])
            
    

    
    


0
['1J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE']
3
['1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05zmYPzblSWc']
7
['1911UP9nR65fxxJO_Na8886TQFBPvMaRstwm4KsoI']
8
['13gTVm1GQEnrQqgc_Q55M8cWyrWrUAmt2gt_v7Ltm18']
10
['102YTIjanaJqeckdzJ_s5dEgUeZQUB1F7G9wWw_6w']
12
['1ywXZQV6GXPPxDAfAJ1Q0AqBD4TTTV214qOvUF79_aI']
13
['1hAdI2qGPe1UIV8qPLBGOk2XLvSwub0LSEQBJSL4XXE']
15
['1gADqw_nKmZGBWvkRGB1YbfxMdiDpCGGFXKMNyLVlO4']
17
['1Zt4SVQIelLhPux24A1S3t5YxIHhp83_b3mAd0AfvA']
27
['1xb7HtpQPHOc3c3j8l4QAq_NwpKFmv_sJHkE1hGbYh4']
28
['1pwQqmPdZnILJUcXY52fY1yOimJRz0lWFK1togzStbM']
31
['1bhzRmCtmrcjraE3Tuw8fv_5jQWGj3NI8INjQgYwH3U']


In [9]:
from google.oauth2 import service_account

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

In [10]:
def get_sheet_data(spreadsheet_id, range_name):
    # SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    # # credentials.json 파일 경로를 지정합니다.
    # creds_filename = './credentials.json'

    # flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
    # # 여기서 인증을 진행합니다. 아래의 메소드 중 하나를 선택하세요.
    # # creds = flow.run_console()
    # creds = flow.run_local_server(port=8080)

    creds = oauth()

    try:
        service = build('sheets', 'v4', credentials=creds)
        # 스프레드시트 ID와 범위(예: "Sheet1!A1:C2")를 지정합니다.
        # result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id).execute()
        result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
        print(result)
        rows = result.get('values', [])

        if not rows:
            print('No data found.')
            return
        print('Name, Major:')
        for row in rows:
            # 행의 각 열을 출력합니다.
            print(f"{row[0]}, {row[1]}")
    except HttpError as error:
        print(f"An error occurred: {error}")

In [11]:
def get_all_sheet_data(spreadsheet_id):
    # SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    # # credentials.json 파일 경로를 지정합니다.
    # creds_filename = './credentials.json'

    # flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
    # # 여기서 인증을 진행합니다. 아래의 메소드 중 하나를 선택하세요.
    # # creds = flow.run_console()
    # creds = flow.run_local_server(port=8080)

    creds = oauth()

    try:
        service = build('sheets', 'v4', credentials=creds)
        # 스프레드시트 ID와 범위(예: "Sheet1!A1:C2")를 지정합니다.
        spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        # result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
        # print("spreadsheet", spreadsheet, "\n")
        sheets = spreadsheet.get('sheets', [])
        sheet2 = sheets[1]
        sheet2_title = sheet2['properties']['title'] 
        result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet2_title).execute()
        sheet2_rows = result.get('values', [])
        sheet2_rows_2 = sheet2_rows[1]
        print("sheet2_rows_2", sheet2_rows_2)
        sheet2_rows_2_current_seller = sheet2_rows_2[1]
        print("sheet2_rows_2_current_seller", sheet2_rows_2_current_seller)
        return sheet2_rows_2_current_seller
        # for sheet in sheets:
        #     title = sheet['properties']['title']
        #     print(f"Sheet title: {title}")
        #     # 각 시트의 첫 번째 셀 데이터를 가져옵니다.
        #     # 전체 데이터를 가져오려면 적절한 범위를 지정하세요.
        #     result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=title).execute()
        #     rows = result.get('values', [])
        #     if not rows:
        #         print('No data found.')
        #         continue
        #     for row in rows:
        #         print(row)  # 각 행의 데이터를 출력합니다.
    except HttpError as error:
        print(f"An error occurred: {error}")

In [13]:
#제피어_연동 1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU
SPREADSHEET_ID = '1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU'
# SPREADSHEET_ID = '1FkXg4GZvM9bnCWwSrgNRlMnNuXymSbe582xbtTQFM1c'
# RANGE_NAME = '시트1!A1:C2'
new_daebong_spreadsheet_title = get_all_sheet_data(SPREADSHEET_ID)
print("new_daebong_spreadsheet_title", new_daebong_spreadsheet_title)

sheet2_rows_2 ['#REF!']


IndexError: list index out of range

In [7]:
# 스프레드시트를 복사하고 새로운 스프레드시트의 ID를 반환하는 함수
def copy_spreadsheet(service, spreadsheet_id, title):
    spreadsheet_body = {
        'properties': {
            'title': title,
        }
    }
    request = service.spreadsheets().create(body=spreadsheet_body)
    new_spreadsheet = request.execute()

    sheets = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute().get('sheets', [])
    for sheet in sheets:
        sheet_id = sheet['properties']['sheetId']
        service.spreadsheets().sheets().copyTo(spreadsheetId=spreadsheet_id, sheetId=sheet_id, body={'destinationSpreadsheetId': new_spreadsheet['spreadsheetId']}).execute()

    return new_spreadsheet['spreadsheetId']

In [21]:
def copy_spreadsheet_and_rename_sheets(service, original_spreadsheet_id, new_title, sheet_names):
    # 새 스프레드시트 생성
    spreadsheet_body = {'properties': {'title': new_title}}
    new_spreadsheet = service.spreadsheets().create(body=spreadsheet_body).execute()
    new_spreadsheet_id = new_spreadsheet['spreadsheetId']

    # 기존 스프레드시트의 시트 정보를 가져옴
    sheets = service.spreadsheets().get(spreadsheetId=original_spreadsheet_id).execute().get('sheets', [])

    # 새 스프레드시트에 시트 복사 및 이름 변경
    print("title", sheets[0]['properties']['title'])
    print("title", sheets[1]['properties']['title'])
    print("title", sheets[2]['properties']['title'])
    for sheet in sheets:
        title = sheet['properties']['title']
        
        if title in sheet_names:
            # 시트 복사
            copied_sheet = service.spreadsheets().sheets().copyTo(
                spreadsheetId=original_spreadsheet_id,
                sheetId=sheet['properties']['sheetId'],
                body={'destinationSpreadsheetId': new_spreadsheet_id}
            ).execute()

            # 복사된 시트의 이름을 원하는 이름으로 변경
            new_sheet_id = copied_sheet['sheetId']
            requests = [{
                'updateSheetProperties': {
                    'properties': {
                        'sheetId': new_sheet_id,
                        'title': sheet_names[title]
                    },
                    'fields': 'title'
                }
            }]
            service.spreadsheets().batchUpdate(spreadsheetId=new_spreadsheet_id, body={'requests': requests}).execute()

    # 새 스프레드시트에서 기본으로 생성된 첫 번째 시트 삭제
    first_sheet_id = new_spreadsheet['sheets'][0]['properties']['sheetId']
    requests = [{'deleteSheet': {'sheetId': first_sheet_id}}]
    service.spreadsheets().batchUpdate(spreadsheetId=new_spreadsheet_id, body={'requests': requests}).execute()

    return new_spreadsheet_id


In [28]:
def copy_entire_spreadsheet(original_spreadsheet_id, copy_title, creds):
    drive_service = build('drive', 'v3', credentials=creds)
    copied_file = {'name': copy_title}
    request = drive_service.files().copy(fileId=original_spreadsheet_id, body=copied_file)
    response = request.execute()
    return response['id']

In [25]:
def apply_formula_to_cells(service, spreadsheet_id, sheet_title):
    range_name = f"{sheet_title}!J11:J"  # J열 11행부터 끝까지
    formula = f'=IF(LEN(E11)=0,"", {new_daebong_spreadsheet_title})'  # 적용할 수식
    value_range_body = {
        'values': [[formula] for _ in range(1000)]  # 예시로 1000행까지 적용, 필요에 따라 조정
    }
    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()

In [24]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet2(service, spreadsheet_id, sheet_title, cell_range, formula):
    range_name = f"{sheet_title}!{cell_range}"  # 지정된 시트의 특정 셀 범위
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [30]:
creds = oauth()
service = build('sheets', 'v4', credentials=creds)

# 1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE 대봉 발주서 파일 ID
original_spreadsheet_id = '1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE'
new_title = new_daebong_spreadsheet_title

# 사용 예시
sheet_names = {
    '발주서': '발주서',
    '누적발주': '누적발주',
    '상품목록': '상품목록'
}


try:
    # new_spreadsheet_id = copy_spreadsheet_and_rename_sheets(service, original_spreadsheet_id, new_title, sheet_names)
    # 스프레드시트 복사
    # new_spreadsheet_id = copy_spreadsheet(service, original_spreadsheet_id, new_title)
    # print(f"New spreadsheet created with ID: {new_spreadsheet_id}")

    new_spreadsheet_id = copy_entire_spreadsheet(original_spreadsheet_id, new_title, creds)
    print(f"New spreadsheet created with ID: {new_spreadsheet_id}")

    # # 수식 적용
    # apply_formula_to_cells(service, new_spreadsheet_id, '발주서')
    # print("Formula applied to cells in the new spreadsheet. 발주서")

    # cell_range = 'A2'  # 수식을 적용할 셀 범위
    # formula = f'=IMPORTRANGE("1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ","\'시트{new_daebong_spreadsheet_title}\'!$A$1:$N$5999")'  # A2 셀에 적용할 수식
    # apply_formula_to_cell_sheet2(service, new_spreadsheet_id, '누적발주', cell_range, formula)
    # print("Formula applied to cells in the new spreadsheet. 누적발주")


except HttpError as error:
        print(f"An error occurred: {error}")


An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE/copy?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


In [1]:
try:
    # 수식 적용
    apply_formula_to_cells(service, new_spreadsheet_id, '발주서')
    print("Formula applied to cells in the new spreadsheet. 발주서")

    cell_range = 'A2'  # 수식을 적용할 셀 범위
    formula = f'=IMPORTRANGE("1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ","\'시트{new_daebong_spreadsheet_title}\'!$A$1:$N$5999")'  # A2 셀에 적용할 수식
    apply_formula_to_cell_sheet2(service, new_spreadsheet_id, '누적발주', cell_range, formula)
    print("Formula applied to cells in the new spreadsheet. 누적발주")


except HttpError as error:
        print(f"An error occurred: {error}")

NameError: name 'HttpError' is not defined